In [1]:
import gurobipy as gp

In [2]:
M = 5
N = 10

cs_i = [ 400 , 100 , 600 , 1000 , 150 ]
ct_jk = [ 2 , 10 , 50 , 3 , 5 , 6 , 15 , 30 , 18 , 30 ]

P_j = [ 100 , 30 , 5 , 130 , 55 , 40 , 15 , 30 , 50 , 10 ]
c_jk = [ 10 , 50 , 500 , 15 , 25 , 30 , 75 , 150 , 90 , 80 ]


es_i = [ 1 , 1 , 1 , 1 , 1 ]
et_jk = [  1 , 1 , 1 , 1 , 1 , 1 , 1 , 1 , 1 , 1 ]

In [3]:
# Rotulos dos clientes
clients = []
for j in range(N):
    clients.append(f"Client_{j}")

# Rotulos das squads
squads = []
for i in range(M):
    squads.append(f"Squad_{i}")

# Rotulos das tarefas
tasks = []
for j in range(N):
    tasks_j  = []
    for k in range(P_j[j]):
        tasks_j.append(f"Task_{j}_{k}")
    
    tasks.append(tasks_j)


In [4]:
# Dicionário das capacidades das Squads
capacities = {}
specialty = {}
for i, cap in enumerate(cs_i):
    label = squads[i]
    capacities[label] = cap
    specialty[label] = es_i[i]

# Dicionário da quantidade de tarefas por cliente
qtd_tasks = {}
for j, qtd in enumerate(P_j):
    label = clients[j]
    qtd_tasks[label] = qtd

# Dicionário de valor, Story Points e especialidade de cada tarefa
values = []
story_points = []
req_expertness = []
for j, qtd in enumerate(P_j):
    value = {}
    story_point = {}
    req_expertise = {}
    for k, label in enumerate(tasks[j]):
        value[label] = c_jk[j]
        story_point[label] = ct_jk[j]
        req_expertise[label] = et_jk[j]
    
    values.append(value)
    story_points.append(story_point)
    req_expertness.append(req_expertise)

In [5]:
# Criando modelo
model = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-21


In [6]:
# Inserindo variáveis de decisão
u_ijk = []
for j in range(N):
    u_ij = model.addVars(squads, tasks[j], vtype=gp.GRB.BINARY) # Ambos os vetores tem apenas rotulos e não valores
    u_ijk.append(u_ij)

# Definindo as restrições

In [7]:
## Garantindo que nenhuma squad tenha sua capacidade máxima exedida
c1 = model.addConstrs(
    gp.quicksum(u_ijk[j][i,k] * story_points[j][k] for j in range(N) for k in tasks[j]) <= capacities[i] for i in squads
)

In [8]:
## Garantindo que a especialidade da squad e tarefa sejam a mesma
c2 = model.addConstr(
    gp.quicksum(u_ijk[j][i,k] * (specialty[i] - req_expertness[j][k]) for j in range(N) for k in tasks[j] for i in squads ) == 0
)

In [9]:
## Garantindo que pelo menos uma tarefa por cliente seja atendida
c3 = model.addConstrs(
    gp.quicksum(u_ijk[j][i,k] for k in tasks[j] for i in squads) >= 1 for j in range(N) 
)

In [10]:
## Garantindo que apenas uma squad pegue a mesma tarefa
c4 = model.addConstrs(
    gp.quicksum(u_ijk[j][i,k] for i in squads) <= 1 for j in range(N) for k in tasks[j] 
)

# Primeira Função Objetivo de Lucro

In [11]:
# Define a função objetivo 1 do lucro
j = 0
model.setObjective(
    gp.quicksum(u_ijk[j][i,k] * values[j][k] for j in range(N) for k in tasks[j] for i in squads ),
    sense=gp.GRB.MAXIMIZE
)

## Colocando uma restrição inútil para obter o valor de f2

In [12]:
## Para calcular o lucro em f2
c5 = model.addConstr(
    gp.quicksum(u_ijk[j][i,k] / P_j[j] for j in range(N) for k in tasks[j] for i in squads ) >= 0
)

## Otimização e resultados

In [13]:
# Executar o modelo
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 482 rows, 2325 columns and 9300 nonzeros
Model fingerprint: 0xfd657522
Variable types: 0 continuous, 2325 integer (2325 binary)
Coefficient statistics:
  Matrix range     [8e-03, 5e+01]
  Objective range  [1e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 12350.000000
Presolve removed 2 rows and 0 columns
Presolve time: 0.02s
Presolved: 480 rows, 2325 columns, 6975 nonzeros
Variable types: 0 continuous, 2325 integer (2325 binary)

Root relaxation: objective 1.243000e+04, 356 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 12430.0000    0    5 12350.0000 12430.0000  0.65%     -    0s
H    0     0               

In [14]:
# Somando a quantidade de tarefas realizadas pela fabrica de software
summation_tasks = 0
for j in range(N):
    for i in squads:
        for k in tasks[j]:
            if round(u_ijk[j][i, k].X) == 1:
                summation_tasks += 1

print(f"Qauntidade de tarefas realizadas: {summation_tasks}")
print(f"Função de satisfação: {-c5.Slack}")

Qauntidade de tarefas realizadas: 359
Função de satisfação: 6.6842307692307665


In [15]:
print(f"Valor obtido: {model.objVal}")

Valor obtido: 12430.0


# Segunda Função Objetivo

In [16]:
# Define a função objetivo 2 de satisfação
j = 0
model.setObjective(
    gp.quicksum(u_ijk[j][i,k] / P_j[j] for j in range(N) for k in tasks[j] for i in squads ),
    sense=gp.GRB.MAXIMIZE
)

## Colocando uma restrição inútil para calcular o valor obtido

In [17]:
## Para calcular o lucro em f2
c6 = model.addConstr(
    gp.quicksum(u_ijk[j][i,k] * values[j][k] for j in range(N) for k in tasks[j] for i in squads ) >= 0
)

## Otimização e resultados

In [18]:
# Executar o modelo
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 483 rows, 2325 columns and 11625 nonzeros
Model fingerprint: 0x65adc072
Variable types: 0 continuous, 2325 integer (2325 binary)
Coefficient statistics:
  Matrix range     [8e-03, 5e+02]
  Objective range  [8e-03, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]

Loaded MIP start from previous solve with objective 6.68423

Presolve removed 3 rows and 0 columns
Presolve time: 0.02s
Presolved: 480 rows, 2325 columns, 6975 nonzeros
Variable types: 0 continuous, 2325 integer (2325 binary)
Found heuristic solution: objective 7.0001748

Root relaxation: objective 8.077778e+00, 778 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.07778    0    7    7.000

In [19]:
# Somando a quantidade de tarefas realizadas pela fabrica de software
summation_tasks = 0
for j in range(N):
    for i in squads:
        for k in tasks[j]:
            if round(u_ijk[j][i, k].X) == 1:
                summation_tasks += 1

print(f"Qauntidade de tarefas realizadas: {summation_tasks}")
print(f"Função de satisfação: {model.objVal}")

Qauntidade de tarefas realizadas: 388
Função de satisfação: 8.07333333333333


In [20]:
print(f"Valor obtido: {-c6.Slack}")

Valor obtido: 11780.0
